In [1]:
%pip install -qU langchain langchain-openai langchain-community langchain-experimental pandas

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import getpass
os.environ["LANGCHAIN_TRACING_V2"] = "true"
if not os.environ.get("LANGCHAIN_API_KEY"):
    os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

 ········


In [3]:
import pandas as pd
df_csv = pd.read_csv('asmr-20240924.csv', encoding='Windows-1252', sep=';', on_bad_lines='skip')
df_csv.columns = df_csv.columns.str.strip('$').str.strip()
print(df_csv.shape)
print(df_csv.columns.tolist())

(9243, 9)
['Code Evamed', 'Motif de demande', 'Code CIS', 'Code CIP', 'Dénomination spécialité', 'Date avis définitif', 'ASMR', 'Valeur ASMR', 'Libellé ASMR$']


In [4]:
import pandas as pd

# Load the Excel file
xls_file = '2023_descriptif-variables_open-medic.xls'

xls_file2 = 'Tableau_moleicules_et_marques_commercialisees_en_France_en_2022.xlsx'
xls2 = pd.read_excel(xls_file2)

# Get the list of sheet names
print("First file preview:")
print(xls2.head())


First file preview:
  Code EPhMRA                        Libellé EPhMRA           Groupe  \
0       L02B9  ANTAG. HORMONES CYTOSTATIQUES AUTRES  JOHNSON&JOHNSON   
1       L02B9  ANTAG. HORMONES CYTOSTATIQUES AUTRES  JOHNSON&JOHNSON   
2        R06A          ANTIHISTAMINIQUES V.GENERALE       UCB PHARMA   
3       N05A1           ANTIPSYCHOTIQUES, ATYPIQUES       PHARMA LAB   
4       N05A1           ANTIPSYCHOTIQUES, ATYPIQUES       PHARMA LAB   

  Laboratoire (constant)      Code EAN  \
0          JANSSEN CILAG  3.400922e+12   
1          JANSSEN CILAG  3.400930e+12   
2             UCB PHARMA  3.400934e+12   
3             PHARMA LAB  3.400950e+12   
4             PHARMA LAB  3.401036e+12   

              Présentation (nom de marque)                   Produit  \
0                  ZYTIGA 250MG CPR FL 120                    ZYTIGA   
1                   ZYTIGA 500MG CPR BT 60                    ZYTIGA   
2           ZYRTEC 10MG/ML BUV FV15ML BT 1                    ZYRTEC   
3   ZY

In [5]:

# Create a dictionary to hold each sheet's DataFrame
sheets_dict = pd.read_excel(xls_file, sheet_name=None)  # sheet_name=None loads all sheets into a dictionary

# Print the keys of the dictionary to confirm DataFrame names (each sheet is a DataFrame)
print("DataFrames created for each sheet:", sheets_dict.keys())


DataFrames created for each sheet: dict_keys(['Variables', 'ATC1', 'ATC2', 'ATC3', 'ATC4', 'ATC5', 'CIP13', 'TOP_GEN', 'GEN_NUM', 'AGE', 'SEXE', 'BEN_REG', 'PSP_SPE'])


In [6]:
from langchain_community.utilities import SQLDatabase
from sqlalchemy import create_engine

engine = create_engine("sqlite:///asmr.db")
print("Engine created for database 'asmr.db'")

Engine created for database 'asmr.db'


In [7]:
# Loop through each sheet in sheets_dict
for sheet_name, df in sheets_dict.items():
    # Clean up the sheet name to use it as a valid table name
    table_name = sheet_name.replace(' ', '_').replace('-', '_').replace('(', '').replace(')', '').lower()
    
    # Load the DataFrame into the database as a separate table
    try:
        df.to_sql(table_name, engine, index=False, if_exists='replace')
        print(f"Successfully added '{sheet_name}' as table '{table_name}' in the database.")
    except Exception as e:
        print(f"Error while adding '{sheet_name}' as table '{table_name}': {e}")


Successfully added 'Variables' as table 'variables' in the database.
Successfully added 'ATC1' as table 'atc1' in the database.
Successfully added 'ATC2' as table 'atc2' in the database.
Successfully added 'ATC3' as table 'atc3' in the database.
Successfully added 'ATC4' as table 'atc4' in the database.
Successfully added 'ATC5' as table 'atc5' in the database.
Successfully added 'CIP13' as table 'cip13' in the database.
Successfully added 'TOP_GEN' as table 'top_gen' in the database.
Successfully added 'GEN_NUM' as table 'gen_num' in the database.
Successfully added 'AGE' as table 'age' in the database.
Successfully added 'SEXE' as table 'sexe' in the database.
Successfully added 'BEN_REG' as table 'ben_reg' in the database.
Error while adding 'PSP_SPE' as table 'psp_spe': Could not reflect: requested table(s) not available in Engine(sqlite:///asmr.db): (psp_spe)


In [8]:
df_csv.to_sql("csv_data", engine, index=False, if_exists='replace')

9243

In [9]:
xls2.to_sql("molecules", engine, index=False, if_exists='replace')

25563

In [10]:
from sqlalchemy import create_engine, inspect

# Create an SQLite engine
engine = create_engine('sqlite:///asmr.db')

# Use the inspector to get a list of all tables in the database
inspector = inspect(engine)
tables = inspector.get_table_names()

# Print the list of tables
print("Existing tables:", tables)



Existing tables: ['PSP_SPE', 'age', 'asmr', 'atc1', 'atc2', 'atc3', 'atc4', 'atc5', 'ben_reg', 'cip13', 'csv_data', 'gen_num', 'gers_12_2022', 'molecules', 'sexe', 'top_gen', 'variables', 'xls_data']


In [11]:
db = SQLDatabase(engine=engine)
print(db.dialect)
print(db.get_usable_table_names())
print(db.run("SELECT * FROM asmr LIMIT 5;"))

sqlite
['PSP_SPE', 'age', 'asmr', 'atc1', 'atc2', 'atc3', 'atc4', 'atc5', 'ben_reg', 'cip13', 'csv_data', 'gen_num', 'gers_12_2022', 'molecules', 'sexe', 'top_gen', 'variables', 'xls_data']
[('$CT-21025$', '$Inscription (CT)$', '$66077218$', '$3400930294611$', '$FEXOFENADINE ARROW 180 mg, comprimé pelliculé$', '$04/09/2024$', '$ASMR$', '$V$', "$Ces spécialités sont des génériques qui n'apportent pas d'amélioration du service médical rendu (ASMR V) par rapport aux spécialités à base de fexofénadine déjà inscrites.$"), ('$CT-21016$', '$Inscription (CT)$', '$64314227$', '$3400955101734$', '$OXYGENE MEDICINAL LINDE HEALTHCARE 200 bar, gaz pour inhalation, en bouteille$', '$28/08/2024$', '$ASMR$', '$V$', "$Cette spécialité est un complément de gamme qui n'apporte pas d'amélioration du service médical rendu (ASMR V) par rapport aux présentations déjà inscrites.$"), ('$CT-21007$', '$Inscription (CT)$', '$60120260$', '$3400930280522$', '$CHLORHEXIDINE LIBERTY PHARMA 0,20 %, solution pour bain 

In [12]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

 ········


In [13]:
from langchain_community.agent_toolkits import create_sql_agent

agent_executor = create_sql_agent(llm, db=db, agent_type="openai-tools", verbose=True)

In [21]:
agent_executor.invoke({"input:can you make me a summary of what you know about the drug zytiga? in 300 characters "})



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


PSP_SPE, age, asmr, atc1, atc2, atc3, atc4, atc5, ben_reg, cip13, csv_data, gen_num, gers_12_2022, molecules, sexe, top_gen, variables, xls_data
Invoking: `sql_db_schema` with `{'table_names': 'molecules'}`



CREATE TABLE molecules (
	"Code EPhMRA" TEXT, 
	"Libellé EPhMRA" TEXT, 
	"Groupe" TEXT, 
	"Laboratoire (constant)" TEXT, 
	"Code EAN" FLOAT, 
	"Présentation (nom de marque)" TEXT, 
	"Produit" TEXT, 
	"CAHT" FLOAT, 
	"Volume" FLOAT, 
	"Date commercialisation" TEXT, 
	"Dosage" TEXT, 
	"Quantité de conditionnement" FLOAT, 
	"Code ATC" TEXT, 
	"Libellé ATC" TEXT, 
	"Groupe Générique (constant)" TEXT, 
	"Générique / Référent (constant)" TEXT, 
	"UN 01/2021" FLOAT, 
	"UN 02/2021" FLOAT, 
	"UN 03/2021" FLOAT, 
	"UN 04/2021" FLOAT, 
	"UN 05/2021" FLOAT, 
	"UN 06/2021" FLOAT, 
	"UN 07/2021" FLOAT, 
	"UN 08/2021" FLOAT, 
	"UN 09/2021" FLOAT, 
	"UN 10/2021" FLOAT, 
	"UN 11/2021" FLOAT, 
	"UN 12/2021" FL

{'input': {'input:can you make me a summary of what you know about the drug zytiga? in 300 characters '},
 'output': 'Zytiga, produced by Janssen Cilag, is an anti-cancer medication that contains abiraterone acetate. It is available in two presentations: 500mg (60 tablets) launched on January 30, 2018, and 250mg (120 tablets), which was released on September 15, 2011. It acts as an antineoplastic hormone antagonist.'}